In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import os
from urlparse import urlparse, urlsplit

In [ ]:
def get_URL(mainURL,year):
    return mainURL+str(year)

def get_link_list(URL):
    '''input one URL link
       output a list of all download links in that page '''
    #get text for that link
    html_text = mysession.get(URL).text
    #create soup object
    soup = BeautifulSoup(html_text, 'html.parser')
    #the link is contained in <a> class = 'episode-download-img' href='...link to mp3'
    links = soup.findAll('a',{'class':'episode-download-img'})
    #extract all the links to mp3
    link_list = [i.get('href') for i in links]
    return link_list

def find_file_name(link,savepath,year):
    '''extract the mp3 file name from the link,
      http://hwcdn.libsyn.com/p/e/5/3/e5328d13367fc4ca/MINI_Belief_in_Santa_Claus.mp3?c_id=8102633...'''
    s = link.split('/')[-1]
    # MINI_Belief_in_Santa_Claus.mp3?c_id=8102633
    filename = re.search('(.*)\?.*',s).group(1)
    # filename to save: 2014_MINI_Belief_in_Santa_Claus.mp3
    return os.path.join(savepath,str(year)+'_'+filename)

def download_mp3(link, filename):
    '''input the link to mp3, and the filename to save
       excute the download'''
    with open(filename,'wb') as f:
        audio_content = mysession.get(link, stream = True)
        for chunk in audio_content.iter_content(chunk_size =255):
            if chunk:
                f.write(chunk)

def download_podcast(mainURL, year_list,savepath):
    for year in year_list:
        # for each year, the page is different
        url= get_URL(mainURL,year)
        # get the list of all links to mp3 in that page
        link_list = get_link_list(url)
        for link in link_list:
            # auto create the filename, adding savepath, and year to savepath
            filename= find_file_name(link,savepath,year)  
            download_mp3(link,filename)

In [ ]:
mysession = requests.Session()
savepath ='/Users/Wei.Zhao/Documents/data skeptic podcast'
mainURL = 'https://dataskeptic.com/podcast/'
year_list = [2014,2015,2016,2017]
download_podcast(mainURL,year_list,savepath)